<a href="https://colab.research.google.com/github/RetoHe/epex-spot-intraday-timeseries-analysis/blob/main/EPEX_Intraday_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download and Data Preparation EPEX Spot Intradaydata
### 1. Download all Intradaytrades for the German Market for 2021
### 2. Merge all files to a single Dataframe
### 3. Prepare the Data
### 4. Export the Dataframe to csv

In [ ]:
! pip install pysftp

In [ ]:
import pysftp
import pandas as pd

In [ ]:
cnopts = pysftp.CnOpts()
cnopts.hostkeys = None

/usr/local/lib/python3.7/dist-packages/pysftp/__init__.py:61: UserWarning: Failed to load HostKeys from /root/.ssh/known_hosts.  You will need to explicitly load HostKeys (cnopts.hostkeys.load(filename)) or disableHostKey checking (cnopts.hostkeys = None).
  warnings.warn(wmsg, UserWarning)


In [ ]:
# Insert your own credentials
sftp = pysftp.Connection(host="ftp.epexspot.com", username="Trading_at_syneco.net", password="syneco01", cnopts=cnopts)

In [ ]:
# move to the correct directory
sftp.chdir("/germany/Intraday Continuous/EOD/Transactions")

In [ ]:
# create folder for the data
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
german_dir = "/germany/Intraday Continuous/EOD/Transactions/"
# download all intradaytrades for 2021
for element in sftp.listdir():
    if element[-3:] == "csv":
        new_path = german_dir + element
        sftp.get(new_path, "data/intradaydata_DE_{}.csv".format(element[29:40]))
    else:
        pass
        

In [ ]:
# Basic syntax:
import glob # Package for Unix-style pathname pattern expansion
import os   # Python operating system interface

directory = 'data/'
# Obtain list of filenames that end in .txt in the directory
all_files = glob.glob(os.path.join(directory, "*.csv"))
# Where os.path.join creates the os-specific path to each file

# Import data however you like. To append data in a single pandas 
# dataframe and a single list, you can do:
master_df = pd.DataFrame()
# map types of the dataframe
column_names = ['Date', 'Market Area Buy', 'Market Area Sell', 'Hour from', 'Hour to', 'Quantity (MW)', 'Price (EUR)', 'Time Stamp', 'Trade ID']
master_df = pd.DataFrame(columns=column_names)
for filename in all_files:
    data = pd.read_csv(filename, dtype={'Date': 'str', 'Market Area Buy': 'str', 'Market Area Sell': 'str', 'Hour from': 'str', 'Hour to': 'str', 'Quantity (MW)': float, 'Price (EUR)': float, 'Time Stamp': 'str', 'Trade ID': 'str'}, skiprows=[0])
    master_df = master_df.append(data)

In [ ]:
master_df = master_df.rename(columns={"Hour from": "Product"})

NameError: ignored

In [ ]:
master_df = master_df.drop(['Hour to'], axis=1)

In [ ]:
# helper function for getting the MWh of the trade
def get_mwh(data):
    work = []
    for i in range(len(data)):
        if "hh" in data["Product"][i]:
            work.append(data["Quantity (MW)"][i] * 0.5)
        elif "qh" in data["Product"][i]:
            work.append(data["Quantity (MW)"][i] * 0.25)
        else:
            work.append(data["Quantity (MW)"][i])

    return work

In [ ]:
# helper function for getting product type
def get_type(data):
    product_type = []
    for i in range(len(data)):
        if "hh" in data["Product"][i]:
            product_type.append("HalfHour")
        elif "qh" in data["Product"][i]:
            product_type.append("QuarterHour")
        else:
            product_type.append("Hour")

    return product_type

In [ ]:
master_df["Work (MWh)"] = get_mwh(master_df)

In [ ]:
master_df["Value (EUR)"] = master_df["Work (MWh)"] * master_df["Price (EUR)"]

In [ ]:
master_df["Product_Type"] = get_type(master_df)

In [ ]:
master_df.to_csv("master_data.csv", index=False)

In [ ]:
from google.colab import files
files.download('master_data.csv') 